# MBSP Cosine Similarity Explorer

This notebook demonstrates interactive cosine similarity search across Sentinel-2 TOA bands. Select a pixel on the MBSP fractional image to compare all other pixels in the scene.

In [ ]:
import datetime as dt
import ee
import geemap
import ipywidgets as widgets

ee.Authenticate()
ee.Initialize()

## Helper Functions

In [ ]:
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    """Mask clouds using the QA60 band."""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    masked = image.updateMask(mask).divide(10000)
    return masked.copyProperties(image, image.propertyNames())

In [ ]:
def mbsp_fractional_image(image: ee.Image, region: ee.Geometry) -> ee.Image:
    num_img = image.select('B11').multiply(image.select('B12'))
    den_img = image.select('B12').multiply(image.select('B12'))
    num_sum = num_img.reduceRegion(reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True)
    den_sum = den_img.reduceRegion(reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True)
    slope = ee.Number(num_sum.get('B11')).divide(ee.Number(den_sum.get('B12')))
    mbsp = (image.select('B12').multiply(slope).subtract(image.select('B11')).divide(image.select('B11')).rename('R'))
    return mbsp.set({'slope': slope})

In [ ]:
BANDS = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']

def cosine_similarity_image(image: ee.Image, sample: ee.Feature) -> ee.Image:
    ref_vals = ee.Image.constant([sample.get(b) for b in BANDS]).rename(BANDS)
    dot = image.select(BANDS).multiply(ref_vals).reduce(ee.Reducer.sum())
    mag1 = image.select(BANDS).pow(2).reduce(ee.Reducer.sum()).sqrt()
    mag2 = ref_vals.pow(2).reduce(ee.Reducer.sum()).sqrt()
    return dot.divide(mag1.multiply(mag2)).rename('similarity')

## Image Selection

In [ ]:
lat, lon = 31.6585, 5.9053
start = dt.date(2019, 10, 1)
end = dt.date(2019, 10, 15)
point = ee.Geometry.Point(lon, lat)
collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate(str(start), str(end))
    .filterBounds(point)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .sort('system:time_start')
    .map(mask_s2_clouds))
images = collection.toList(collection.size())
count = images.size().getInfo()
print(f'Found {count} images')

## Interactive Map

In [ ]:
if count:
    region = point.buffer(1000).bounds()
    img = ee.Image(images.get(0))
    date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    frac = mbsp_fractional_image(img, region)
    m = geemap.Map(center=(lat, lon), zoom=12)
    m.addLayer(img.select(['B4','B3','B2']), {'min':0, 'max':0.3}, 'RGB', False)
    m.addLayer(frac, {'min':-0.05,'max':0.05,'palette':['blue','white','red']}, 'Fractional', True)
    frac_layer = m.layers[-1]
    m.sim_layer = None

    def handle_click(**kwargs):
        if kwargs.get('type') == 'click':
            latc, lonc = kwargs.get('coordinates')
            pt = ee.Geometry.Point(lonc, latc)
            sample = img.select(BANDS).sample(pt, scale=20).first()
            if sample:
                sim = cosine_similarity_image(img, sample)
                m.addLayer(sim, {'min':0,'max':1,'palette':['white','green']}, 'Similarity', False)
                m.sim_layer = m.layers[-1]

    m.on_interaction(handle_click)
    display(m)
    toggle = widgets.ToggleButtons(options=['Fractional','Similarity'], description='View:')

    def switch(change):
        if m.sim_layer is None:
            return
        frac_layer.visible = change['new'] == 'Fractional'
        m.sim_layer.visible = change['new'] == 'Similarity'

    toggle.observe(switch, 'value')
    display(toggle)